### 1. 5 diferenças entre o AdaBoost e o GBM

**Método de combinação:** <br> 
O AdaBoost combina várias versões de um único modelo fraco, os chamados stumps. O GBM, por outro lado, cria modelos de árvore de decisão.

**Pesos das amostras** <br> 
No AdaBoost, as amostras são ponderadas de acordo com sua dificuldade de classificação. As amostras mal classificadas recebem um peso maior, permitindo que os modelos subsequentes se concentrem nelas. No GBM, não há ponderação de amostras, pois o foco está na redução dos erros residuais.

**Abordagem de otimização** <br> 
O AdaBoost utiliza uma abordagem de otimização por gradiente descendente, onde o objetivo é minimizar a função de perda ajustando os pesos dos modelos fracos. O GBM também utiliza uma abordagem de otimização por gradiente descendente, mas em vez de ajustar os pesos, ele ajusta os parâmetros dos modelos de árvore de decisão.

**Maneira de criar os modelos** <br> 
No AdaBoost, os modelos fracos são criados sequencialmente, onde cada novo modelo é treinado para corrigir os erros cometidos pelos modelos anteriores. No GBM, os modelos de árvore de decisão são criados de forma iterativa, onde cada nova árvore é treinada para minimizar os erros residuais.

**Robustez a ruídos** <br> 
O AdaBoost é sensível a ruídos e outliers, pois os modelos subsequentes se concentram em corrigir os erros cometidos pelos modelos anteriores. O GBM é mais robusto a ruídos e outliers, pois a abordagem de otimização por gradiente descendente ajuda a reduzir o impacto desses valores atípicos.


### 2. Exemplo do GradientBoostingRegressor

In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)

X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]

est = GradientBoostingRegressor(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=1, 
    random_state=0,
    loss='squared_error'
).fit(X_train, y_train)

mean_squared_error(y_test, est.predict(X_test))

5.009154859960321

### 3. Hiperparâmetros importantes no GradientBoostingRegressor

**n_estimators:** <br> 
Este parâmetro define o número de árvores de decisão que serão criadas. Um valor maior para n_estimators pode aumentar a capacidade do modelo, mas também pode levar a um maior tempo de treinamento. 

**learning_rate** <br> 
O learning_rate controla a taxa de aprendizado do algoritmo. Um valor menor para learning_rate reduz a contribuição de cada árvore, tornando o processo de aprendizado mais lento, mas potencialmente mais preciso. Por outro lado, um valor maior para learning_rate aumenta a contribuição de cada árvore, tornando o processo de aprendizado mais rápido, mas potencialmente menos preciso.

**max_depth** <br> 
O max_depth define a profundidade máxima de cada árvore de decisão. Um valor maior para max_depth permite que as árvores sejam mais complexas e capazes de se ajustar aos dados de treinamento com mais detalhes. No entanto, um valor muito alto pode levar a um overfitting. 

**min_samples_split** <br> 
O min_samples_split define o número mínimo de amostras necessárias para dividir um nó interno durante a construção de uma árvore. Um valor maior para min_samples_split pode evitar o overfitting, pois requer um número mínimo de amostras em um nó para que a divisão ocorra. No entanto, um valor muito alto pode levar a uma árvore que não captura informações importantes nos dados. 

**loss** <br> 
O loss define a função de perda utilizada para otimizar o modelo. O GradientBoostingRegressor suporta várias funções de perda, como squared_error, absolute_error, huber e quantile. A escolha da função de perda depende do problema específico e das características dos dados.

### 4. Uso do GridSearchCV para encontrar os melhores hiperparâmetros

In [2]:
from sklearn.model_selection import GridSearchCV

In [4]:
parameters = {
    'n_estimators': [100, 150, 200, 250],
    'learning_rate': [0.1, 0.2, 0.3],
    'max_depth': [1, 2],
    'min_samples_split': [2, 3],
    'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'random_state': [0],
}

In [5]:
%%time

grid_rf = GridSearchCV(
    estimator = GradientBoostingRegressor(),
    param_grid = parameters,
    cv = 3,
    n_jobs = -1
)

grid_rf.fit(X_train, y_train)

CPU times: user 1.06 s, sys: 110 ms, total: 1.17 s
Wall time: 43.1 s


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'loss': ['squared_error', 'absolute_error', 'huber',
                                  'quantile'],
                         'max_depth': [1, 2], 'min_samples_split': [2, 3],
                         'n_estimators': [100, 150, 200, 250],
                         'random_state': [0]})

- **Parâmetros testados:**

In [9]:
import pandas as pd
pd.DataFrame(grid_rf.cv_results_['params'])

,learning_rate,loss,max_depth,min_samples_split,n_estimators,random_state
0,0.1,squared_error,1,2,100,0
1,0.1,squared_error,1,2,150,0
2,0.1,squared_error,1,2,200,0
3,0.1,squared_error,1,2,250,0
4,0.1,squared_error,1,3,100,0
...,...,...,...,...,...,...
187,0.3,quantile,2,2,250,0
188,0.3,quantile,2,3,100,0
189,0.3,quantile,2,3,150,0
190,0.3,quantile,2,3,200,0


- **Melhores parâmetros:**

In [10]:
print(grid_rf.best_estimator_)

GradientBoostingRegressor(learning_rate=0.2, max_depth=2, min_samples_split=3,
                          n_estimators=200, random_state=0)


### 5. Diferença entre o Gradient Boosting e Stochastic Gradient Boosting

Pensando no nome dado, a diferença está na aleatoriedade. No GBM, cada árvore de decisão é construída utilizando todo o conjunto de dados de treinamento, sem amostragem aleatória. Por outro lado, no Stochastic Gradient Boosting, cada árvore de decisão é construída utilizando uma amostra aleatória do conjunto de dados de treinamento. Essa amostragem aleatória é realizada com substituição, o que significa que uma mesma amostra pode ser selecionada várias vezes ou até mesmo não ser selecionada.